# OCI Data Science Model Deployment Endpoint

This notebook provides a quick overview for getting started with OCI Data Science Model Deployment [chat models](/docs/concepts/#chat-models).

[OCI Data Science](https://docs.oracle.com/en-us/iaas/data-science/using/home.htm) is a fully managed and serverless platform for data science teams to build, train, and manage machine learning models in the Oracle Cloud Infrastructure.

This notebooks goes over how to use a chat model hosted on a [OCI Data Science Model Deployment](https://docs.oracle.com/en-us/iaas/data-science/using/model-dep-about.htm).

To authenticate, [oracle-ads](https://accelerated-data-science.readthedocs.io/en/latest/user_guide/cli/authentication.html) has been used to automatically load credentials for invoking endpoint.


## Overview
### Integration details

| Class | Package | Local | Serializable | [JS support](https://js.langchain.com/v0.2/docs/integrations/chat/oci_generative_ai) | Package downloads | Package latest |
| :--- | :--- | :---: | :---: |  :---: | :---: | :---: |
| [ChatOCIModelDeploymentEndpoint](https://api.python.langchain.com/en/latest/chat_models/langchain_community.chat_models.oci_data_science_model_deployment_endpoint.ChatOCIModelDeploymentEndpoint.html) | [langchain-community](https://api.python.langchain.com/en/latest/community_api_reference.html) | ❌ | ❌ | ❌ | ![PyPI - Downloads](https://img.shields.io/pypi/dm/langchain-oci-model-deployment?style=flat-square&label=%20) | ![PyPI - Version](https://img.shields.io/pypi/v/langchain-oci-model-deployment?style=flat-square&label=%20) |

### Model features
| [Tool calling](/docs/how_to/tool_calling/) | [Structured output](/docs/how_to/structured_output/) | JSON mode | [Image input](/docs/how_to/multimodal_inputs/) | Audio input | Video input | [Token-level streaming](/docs/how_to/chat_streaming/) | Native async | [Token usage](/docs/how_to/chat_token_usage_tracking/) | [Logprobs](/docs/how_to/logprobs/) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: | :---: |
| ❌ | ❌ | ✅  | ❌ | ❌ | ❌ | ✅ | ✅  | ✅ | ❌ | 

## Prerequisite

### Deploy model
Check [Oracle GitHub samples repository](https://github.com/oracle-samples/oci-data-science-ai-samples/tree/main/model-deployment/containers/llama2) on how to deploy your chat model on OCI Data Science Model deployment.

### Policies
Make sure to have the required [policies](https://docs.oracle.com/en-us/iaas/data-science/using/model-dep-policies-auth.htm#model_dep_policies_auth__predict-endpoint) to access the OCI Data Science Model Deployment endpoint.

## Set up

To access OCI Data Science Model Deployment models you'll need to create an OCI account, set up credentials, and install required packages.

### Authentication

You can set authentication through either ads or environment variables. When you are working in OCI Data Science Notebook Session, you can leverage resource principal to access other OCI resources. Check out [here](https://accelerated-data-science.readthedocs.io/en/latest/user_guide/cli/authentication.html) to see more options. 


## Installation

In [ ]:
%pip install -qU langchain-community oracle-ads

## Instantiation

Now we can instantiate our model object and generate chat completions:

In [ ]:
import ads
from langchain_community.chat_models import ChatOCIModelDeploymentEndpoint

# Set authentication through ads
# Use resource principal are operating within a
# OCI service that has resource principal based
# authentication configured
ads.set_auth("resource_principal")

# Create an instance of OCI Model Deployment Endpoint
# Replace the endpoint uri and model name with your own
chat = ChatOCIModelDeploymentEndpoint(
    model="odsc-llm",
    endpoint="https://modeldeployment.us-ashburn-1.oci.customer-oci.com/<ocid>/predict",
    auth=ads.common.auth.default_signer(),
    streaming=False,
    max_retries=3,
    stop=None,
    model_kwargs={
        "temperature": 0.2,
        "max_tokens": 512,
    },  # model params...
)

## Invocation

In [ ]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]

ai_msg = chat.invoke(messages)
ai_msg

In [ ]:
print(ai_msg.content)

## Chaining

We can chain our model with a prompt template like so:

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | chat
chain.invoke(
    {
        "input_language": "English",
        "output_language": "German",
        "input": "I love programming.",
    }
)

## Asynchronous calls

We can make asynchronous calls via the Runnables [Async Interface](/docs/concepts#interface).

In [ ]:
# for running these examples in the notebook:
import nest_asyncio
import asyncio

from langchain_community.chat_models import ChatOCIModelDeploymentEndpoint

nest_asyncio.apply()

In [ ]:
system = (
    "You are a helpful assistant that translates {input_language} to {output_language}."
)
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chat = ChatOCIModelDeploymentEndpoint(
    endpoint="https://modeldeployment.us-ashburn-1.oci.customer-oci.com/<ocid>/predict"
)
chain = prompt | chat

asyncio.run(
    chain.ainvoke(
        {
            "input_language": "English",
            "output_language": "Chinese",
            "text": "I love programming",
        }
    )
)

## Streaming calls

We can also stream outputs via the `stream` method:

In [ ]:
import sys
import ads

from langchain_community.chat_models import ChatOCIModelDeploymentEndpoint

ads.set_auth("security_token", profile="custboat")
prompt = ChatPromptTemplate.from_messages(
    [("human", "List out the 5 most populous countries in the world")]
)

chat = ChatOCIModelDeploymentEndpoint(
    endpoint="https://modeldeployment.us-ashburn-1.oci.customer-oci.com/<ocid>/predict"
)

chain = prompt | chat

for chunk in chain.stream({}):
    sys.stdout.write(chunk.content)
    sys.stdout.flush()